In [20]:
# GameData分析用のユーティリティ関数群

import sqlite3
from datetime import datetime, timedelta
import pandas as pd
from typing import List, Optional
import warnings

DB_PATH = '/Users/hide/Documents/sqlite3/gamehard.db'


In [ ]:
def load_gamehard_weekly() -> pd.DataFrame:
    """
    sqlite3を使用してデータベースからハードウェア販売データを読み込む関数。
    日付関係のカラムをdatetime64[ns]型に変換して返す。
    
    Returns:
        pd.DataFrame: ハードウェア販売データのDataFrame。
    """
    # SQLite3データベースに接続
    conn = sqlite3.connect(DB_PATH)
    # SQLクエリを実行してデータをDataFrameに読み込む
    query = "SELECT id, report_date, period_date as period, hw, units FROM gamehard_weekly ORDER BY id;"
    df = pd.read_sql_query(query, conn)
    
    # 接続を閉じる
    conn.close()

    # 日付をdatetime64[ns]型に変換
    df['report_date'] = pd.to_datetime(df['report_date'])
    df.set_index('report_date', inplace=True)

    # 不要なカラム(id)を削除
    df.drop(columns=['id'], inplace=True)

    return df


df_weekly = load_gamehard_weekly()
df_weekly["cumsum"] = df_weekly.groupby("hw")["units"].cumsum()
df_weekly


,period,hw,units,cumsum
report_date,,,,
1998-10-18,7,GB,10867,10867
1998-10-18,7,N64,2965,2965
1998-10-18,7,PS,20918,20918
1998-10-18,7,SATURN,858,858
1998-10-25,7,GB,163017,173884
...,...,...,...,...
2025-08-03,7,NS2,62733,1677123
2025-08-03,7,NSW,18637,35869618
2025-08-03,7,PS4,19,9490707


In [ ]:
def load_gamehard_info() -> pd.DataFrame:
    conn = sqlite3.connect(DB_PATH)
    query = "SELECT id, launch_date, maker_name, full_name FROM gamehard_info ORDER BY id;"
    df = pd.read_sql_query(query, conn)

    conn.close()

    df["launch_date"] = pd.to_datetime(df["launch_date"])
    df.set_index('id', inplace=True)
    return df

df_gameinfo = load_gamehard_info()



In [47]:
df_weekly2 = df_weekly.merge(df_gameinfo, left_on="hw", right_index=True)

# df_weekly2のperiodindexとlaunch_dateを比較して、delta_year, delta_month, delta_weekを作り出す
# delta_yearはカレンダー年の比較、delta_monthはカレンダー月の比較、delta_weekはカレンダー週の比較
df_weekly2["delta_year"] = df_weekly2.index.year - df_weekly2["launch_date"].dt.year
df_weekly2["delta_month"] = (df_weekly2.index.year - df_weekly2["launch_date"].dt.year) * 12 + df_weekly2.index.month - df_weekly2["launch_date"].dt.month
df_weekly2["delta_week"] = (df_weekly2.index - df_weekly2["launch_date"]).dt.days // 7


df_weekly2.sort_index(inplace=True)
df_weekly2

,period,hw,units,cumsum,launch_date,maker_name,full_name,delta_year,delta_month,delta_week
report_date,,,,,,,,,,
1998-10-18,7,GB,10867,10867,1989-04-21,Nintendo,GAME BOY,9,114,495
1998-10-18,7,PS,20918,20918,1994-12-03,SONY,PlayStation,4,46,202
1998-10-18,7,N64,2965,2965,1996-06-23,Nintendo,NINTENDO64,2,28,121
1998-10-18,7,SATURN,858,858,1994-11-22,SEGA,SEGA SATURN,4,47,203
1998-10-25,7,PS,21151,42069,1994-12-03,SONY,PlayStation,4,46,203
...,...,...,...,...,...,...,...,...,...,...
2025-08-03,7,PS5,7141,6946379,2020-11-12,SONY,PlayStation5,5,57,246
2025-08-03,7,PS4,19,9490707,2014-02-22,SONY,PlayStation4,11,138,597
2025-08-03,7,NSW,18637,35869618,2017-03-03,Nintendo,Nintendo Switch,8,101,439


In [50]:
df_weekly3 = df_weekly2.copy()
df_weekly3["year"] = df_weekly3.index.year
df_weekly3["month"] = df_weekly3.index.month
df_weekly3["week"] = (df_weekly3.index.day // 7) + 1

df_weekly3.tail(30)

,period,hw,units,cumsum,launch_date,maker_name,full_name,delta_year,delta_month,delta_week,year,month,week
report_date,,,,,,,,,,,,,
2025-06-29,7,NS2,161021,1169684,2025-06-05,Nintendo,Nintendo Switch2,0,0,3,2025,6,5
2025-06-29,7,PS4,23,9490597,2014-02-22,SONY,PlayStation4,11,136,592,2025,6,5
2025-06-29,7,XSX,353,680598,2020-11-10,Microsoft,Xbox Series X|S,5,55,241,2025,6,5
2025-06-29,7,NSW,15323,35793449,2017-03-03,Nintendo,Nintendo Switch,8,99,434,2025,6,5
2025-06-29,7,PS5,7333,6910279,2020-11-12,SONY,PlayStation5,5,55,241,2025,6,5
2025-07-06,7,PS4,21,9490618,2014-02-22,SONY,PlayStation4,11,137,593,2025,7,1
2025-07-06,7,XSX,529,681127,2020-11-10,Microsoft,Xbox Series X|S,5,56,242,2025,7,1
2025-07-06,7,NS2,128643,1298327,2025-06-05,Nintendo,Nintendo Switch2,0,1,4,2025,7,1
2025-07-06,7,NSW,17712,35811161,2017-03-03,Nintendo,Nintendo Switch,8,100,435,2025,7,1
